# Tokenizer
Tokenizer（分词器）将文本分割成单词或子词并转化为数组编号

## 训练简单分词器
### 初始化
使用字节对编码（BPE）算法

In [1]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer
)
tokenizer = Tokenizer(models.BPE())

tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

### 定义特殊标记
数据集中存在一些不希望被分词的特殊标记，应定义为特殊标记，防止出现错误的分词情况

In [2]:
special_tokens = ["<unk>", "<s>", "</s>"]

trainer = trainers.BpeTrainer(
    vocab_size=256,
    special_tokens=special_tokens,
    show_progress=True,
    # 使用 ByteLevel 预分词器的初始字母表。
    # 作用: 提供一个基础的字符集（例如 ASCII 字符集），确保分词器能够处理所有可能的字符。
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet(),
)

### 读取数据
使用JSON Lines（jsonl）格式存储Tokenizer训练数据，分词器内置的训练函数要求训练数据以迭代器的形式传入，因此先获取一个数据读取的生成器

In [3]:
import json
import os

def read_texts_from_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line)
            yield data['text']

# 打印当前文件路径
print("Current file path:", os.getcwd())
data_path = '../toydata/tokenizer_data.jsonl'
data_iterator = read_texts_from_jsonl(data_path)
print(f"Row 1: {next(data_iterator)}")

Current file path: e:\breakdown-minimind\script
Row 1: <s>近年来，人工智能技术迅速发展，深刻改变了各行各业的面貌。机器学习、自然语言处理、计算机视觉等领域的突破性进展，使得智能产品和服务越来越普及。从智能家居到自动驾驶，再到智能医疗，AI的应用场景正在快速拓展。随着技术的不断进步，未来的人工智能将更加智能、更加贴近人类生活。</s>


### 开始训练
使用分词器内置函数 `tokenizer.train_from_iterator` 来训练分词器

In [4]:
tokenizer.train_from_iterator(data_iterator, trainer=trainer)

### 设置解码器

In [5]:
tokenizer.decoder = decoders.ByteLevel()

然后检查特殊标记是否正确被处理

In [6]:
assert tokenizer.token_to_id("<unk>") == 0
assert tokenizer.token_to_id("<s>") == 1
assert tokenizer.token_to_id("</s>") == 2

### 将训练好的分词器保存

In [7]:
tokenizer_dir = '../model/toy_tokenizer'
os.makedirs(tokenizer_dir, exist_ok=True)
tokenizer.save(os.path.join(tokenizer_dir, 'tokenizer.json'))
tokenizer.model.save(tokenizer_dir)

['../model/toy_tokenizer\\vocab.json', '../model/toy_tokenizer\\merges.txt']

### 创建配置文件

In [8]:
config = {
    "add_bos_token": False,
    "add_eos_token": False,
    "add_prefix_space": False,
    "added_tokens_decoder": {
        "0": {
            "content": "<unk>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True,
        },
        "1": {
            "content": "<s>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True,
        },
        "2": {
            "content": "</s>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True,
        },
    },
    "additional_special_tokens": [],
    "bos_token": "<s>",
    "clean_up_tokenization_spaces": False,
    "eos_token": "</s>",
    "legacy": True,
    "model_max_length": 32768,
    "pad_token": "<unk>",
    "sp_model_kwargs": {},
    "spaces_between_special_tokens": False,
    "tokenizer_class": "PreTrainedTokenizerFast",
    "unk_token": "<unk>",
    "chat_template": "{{ '<s>' + messages[0]['text'] + '</s>' }}",
}

with open(os.path.join(tokenizer_dir, "tokenizer_config.json"), 'w', encoding='utf-8') as f:
    json.dump(config, f, ensure_ascii=False, indent=4)

print("Tokenizer training and saving completed successfully.")

Tokenizer training and saving completed successfully.


现在已经训练了一个简单的分词器，并将其进行保存，接下来，我们试着加载它，并使用其帮助我们对文本进行编码

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("../model/toy_tokenizer")
message = [{"text": "失去的东西就要学者去接受，学着放下。"}]
new_message = tokenizer.apply_chat_template(conversation=message, tokenize=False)
print(f"原始文本：{message}")
print(f"应用聊天模板后的文本：{new_message}")

e:\miniconda3\envs\MiniMind\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


原始文本：[{'text': '失去的东西就要学者去接受，学着放下。'}]
应用聊天模板后的文本：<s>失去的东西就要学者去接受，学着放下。</s>


In [10]:
print(f"分词器词表大小：{tokenizer.vocab_size}")

分词器词表大小：259


In [11]:
model_inputs = tokenizer(new_message)
print(f"查看分词结果：\n{model_inputs}")

查看分词结果：
{'input_ids': [1, 164, 100, 112, 164, 239, 122, 166, 251, 229, 163, 119, 253, 167, 101, 126, 164, 111, 112, 167, 102, 226, 164, 258, 102, 167, 225, 230, 164, 239, 122, 165, 239, 101, 164, 240, 248, 174, 123, 237, 164, 258, 102, 166, 254, 225, 165, 245, 125, 163, 119, 236, 162, 225, 227, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [12]:
response = tokenizer.decode(model_inputs['input_ids'], skip_special_tokens=False)
print(f"对分词结果进行解码：\n{response}")

对分词结果进行解码：
<s>失去的东西就要学者去接受，学着放下。</s>
